Code to explore 3-point data

In [ ]:
import numpy as np
import h5py
import time
import os
import matplotlib.pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
import mpl_toolkits.mplot3d.axes3d as p3
import matplotlib.animation as animation
%matplotlib notebook
from collections import Counter
import math
import cv2

# Import data

In [ ]:
file_paths = ["../predictions/20201119/bottomup/pred_FishTank20200416_160648/results/FishTank20200416_160648/FishTank20200416_160648.h5",
              "../predictions/20201119/bottomup/pred_FishTank20200413_154621/results/FishTank20200413_154621/FishTank20200413_154621.h5",
              "../predictions/20201120/pred_FishTank20200414_154100/results/FishTank20200414_154100/FishTank20200414_154100.h5",
              "../predictions/20201120/pred_FishTank20200415_154234/results/FishTank20200415_154234/FishTank20200415_154234.h5",
              "../predictions/20201120/pred_FishTank20200417_154139/results/FishTank20200417_154139/FishTank20200417_154139.h5",
              "../predictions/20201120/pred_FishTank20200418_160144/results/FishTank20200418_160144/FishTank20200418_160144.h5",
              "../predictions/20201120/pred_FishTank20200419_151352/results/FishTank20200419_151352/FishTank20200419_151352.h5",
              "../predictions/20201120/pred_FishTank20200419_173651/results/FishTank20200419_173651/FishTank20200419_173651.h5",
              "../predictions/20201120/pred_FishTank20200420_153056/results/FishTank20200420_153056/FishTank20200420_153056.h5",
              "../predictions/20201120/pred_FishTank20200420_175237/results/FishTank20200420_175237/FishTank20200420_175237.h5",
             ]

data_3D = []
data_3D_linear_filled = []
data_3D_filtered = []
for file_path in file_paths:
    with h5py.File(file_path, 'r') as f:
        data_3D.append(np.copy(f['tracks_3D']))
        data_3D_linear_filled.append(np.copy(f['tracks_3D_linear_filtered']))
        data_3D_filtered.append(np.copy(f['tracks_3D_no_outliers_50']))

In [ ]:
for k, dataset in enumerate(data_3D):
    data_test = np.copy(dataset)
    deleted = 0
    for bodypart in range(3):
        for coordinate in range(3):
            differences = []
            datapart = data_test[:, 0, bodypart, coordinate]
            for i in np.arange(1, len(datapart)):
                if np.isnan(datapart[i]):
                    continue

                j = i-1
                before = datapart[j]
                while np.isnan(before):
                    j -= 1
                    before = datapart[j]

                if abs(datapart[j] - datapart[i])*100/(i-j) > 50:
                    datapart[i] = np.nan
                    deleted += 1

            data_test[:, 0, bodypart, coordinate] = datapart
    print(deleted)
    
    with h5py.File(file_paths[k], 'a') as hf:
        del hf['tracks_3D_no_outliers_50']
        hf.create_dataset('tracks_3D_no_outliers_50', data=data_test)

# Missing data check

In [ ]:
def missing_data(dataset):
    missing = np.zeros(len(dataset))
    missing_bdps = [0,0,0]
    missed_frames = []

    for i, frame in enumerate(dataset):
        for j, bdp in enumerate(frame[0]):
            if np.any(np.isnan(bdp)):
                missing_bdps[j] += 1
                missing[i] += 1
                missed_frames.append(i)
                
    print("missing [head, thorax, tail]:", missing_bdps)
    print("total missing bodypoints:", sum(missing_bdps))
    print(Counter(missing))
    
    unique_missing_frames = sorted(set(missed_frames))
    missed_frame_sequences = []
    if len(unique_missing_frames) > 0:
        sequence = [unique_missing_frames[0]]
    else:
        print("no missing frames")
        return
    
    for frame in unique_missing_frames[1:]:
        if frame - sequence[-1] < 2:
            sequence.append(frame)
        else:
            missed_frame_sequences.append(sequence)
            sequence = [frame]
    missed_frame_sequences.append(sequence)
    sequence_lengths = [len(i) for i in missed_frame_sequences]
    sequence_lengths, missed_frame_sequences = zip(*sorted(zip(sequence_lengths, missed_frame_sequences)))
    print([i for i in sequence_lengths if i > 1])
    
    return

In [ ]:
def nan_to_averaged_linear(data):
    
    # look for nan
    for i in range(len(data)):
        if np.isnan(data[i]):
            
            # pick nearest non-nan values
            value_0 = data[i - 1]
            j = i
            while np.isnan(data[j]):
                j += 1
                
                # if nan at end of file
                if j > len(data):
                    j = i - 1
                    break
                    
            value_1 = data[j]
            
            # calculate linear gradient
            difference = value_1 - value_0
            gradient = difference / (j - (i - 1))
        
            # fill in nans
            for k in range(i, j):
                data[k] = value_0 + gradient * (k - (i - 1))
            
    return np.array(data)

def no_nans_no_more(dataset):
    for i in range(3):
        for j in range(3):
            dataset[:, 0, i, j] = nan_to_averaged_linear(dataset[:, 0, i, j])
    return dataset

# X, Y, Z - Positions thorax

In [ ]:
def scatterplot_xyz(data, names):
    fig, axes = plt.subplots(1, len(names), sharey=True, squeeze=False)
    
    for i in range(len(names)):
        axes[0, i].scatter(data[-1], data[i], s=1)
        axes[0, i].set_title(names[i])
        axes[0, i].set_xlabel('frame number')
        axes[0, i].set_ylabel('position (cm)')
        
    plt.show()
    return

def plot_scatter_data(data):
    plt.figure()
    plt.scatter(np.linspace(0, len(data)-1, len(data)), data)
    plt.show()
    return

def histograms_xyz(data, names):
    fig, axes = plt.subplots(1, len(names), sharey=True, squeeze=False)
    
    for i in range(len(names)):
        axes[0, i].hist(data[i], density=True, bins=40)
        axes[0, i].set_xlim(0,40)
        axes[0, i].set_title(names[i])
        axes[0, i].set_xlabel('position (cm)')
        axes[0, i].set_ylabel('denisty')
        
    plt.show()
    return

def heatmap_plot(xd, yd, title):
    # Create heatmap
    xd_new = []
    yd_new = []
    for i in range(len(yd)):
        if np.any(np.isnan([xd[i], yd[i]])):
            continue
        else:
            xd_new.append(xd[i])
            yd_new.append(yd[i])
    heatmap, xedges, yedges = np.histogram2d(yd_new, xd_new, bins=(64,64))

    extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]

    # Plot heatmap
    plt.figure()
    plt.clf()
    plt.title(title)
    plt.ylabel('y')
    plt.xlabel('x')
    plt.ylim(0, 40)
    plt.xlim(0, 40)
    plt.imshow(heatmap, extent=extent, origin='lower')
    plt.colorbar()
    plt.show()
    
    return

def bodypoint_positions(dataset, names=['x', 'y', 'z'], plots=[True, True, True], bodypoint="thorax", frame_range=None):
    
    # take coordinates for one bodypoint
    bodypoints = {"head": 0, "thorax": 1, "tail": 2}
    if frame_range:
        coords = dataset[frame_range[0]:frame_range[-1], 0, bodypoints[bodypoint], :]
        linsp = np.linspace(frame_range[0], frame_range[-1], frame_range[-1] - frame_range[0])
    else:
        coords = dataset[:, 0, bodypoints[bodypoint], :]
        linsp = np.linspace(0, len(coords)-1, len(coords))
        
    x = coords[:, 0]
    y = coords[:, 1]
    z = coords[:, 2]
    
    data_plots = []
    for i in names:
        if i == 'x':
            data_plots.append(x)
        elif i == 'y':
            data_plots.append(y)
        elif i == 'z':
            data_plots.append(z)
    data_plots.append(linsp)
    
    if plots[0]:
        scatterplot_xyz(data_plots, names)
    if plots[1]:
        histograms_xyz(data_plots, names)
    if plots[2]:
        heatmap_plot(x, y, bodypoint + ' xy heatmap (specfic range: {})'.format(frame_range))
    
    return

# Z-specific xy heatmaps & histograms

In [ ]:
def get_xy_for_z(data, zmin, zmax):
    x_data = []
    y_data = []
    for coords in data:
        x_coord = coords[0]
        y_coord = coords[1]
        z_coord = coords[2]
        if z_coord < zmax and z_coord > zmin:
            x_data.append(x_coord)
            y_data.append(y_coord)
    return x_data, y_data

def z_specific_xy(dataset, zstart, zstop, stepsize):
    while zstart < zstop:
        x_data, y_data = get_xy_for_z(dataset, zstart, zstart + stepsize)
        heatmap_plot(x_data, y_data, "xy for z between {} and {}".format(zstart, zstart + stepsize))
        histograms_xyz([x_data, y_data], ['x for z [{},{}]'.format(zstart, zstart + stepsize), 'y for z [{},{}]'.format(zstart, zstart + stepsize)])
        zstart = zstart + stepsize
    return

# xy direction upperbody angle

In [ ]:
def calc_angle_xpos(vector):
    return math.atan2(vector[1], vector[0]) * 180 / math.pi

def calculate_angles(data_1, data_2, axes):
    angles = []
    
    # A_vector = (x1 - x2)i + (y1 - y2)j
    for i in range(len(data_1)):
        x1 = data_1[i][axes[0]]
        x2 = data_2[i][axes[0]]
        y1 = data_1[i][axes[1]]
        y2 = data_2[i][axes[1]]
        
        angles.append(calc_angle_xpos([x1 - x2, y1 - y2]))
        
    return angles

def angle_directional(data_1, data_2, axes):
    angles = []
    for i in range(len(data_1)):
        x1 = data_1[i][axes[0]]
        x2 = data_2[i][axes[0]]
        y1 = data_1[i][axes[1]]
        y2 = data_2[i][axes[1]]
        angle = np.arctan2(y1 - y2, x1 - x2) * 180 / np.pi
        if angle < 0:
            angle = 360 + angle
        angles.append(angle)
    return angles

def angle_add_xy(data):
    angles = []
    pref_vector = [1, 0]
    for i in range(len(data)):
        x1 = data[i][0][0][0]
        x2 = data[i][0][1][0]
        y1 = data[i][0][0][1]
        y2 = data[i][0][1][1]
        vector = [x1-x2,y1-y2]
        
        angle_1 = calc_angle_xpos(vector)
        angle_2 = calc_angle_xpos(pref_vector)
        
        angle = np.mod(angle_1 - angle_2, 360)
        
        if angle > 180:
            angle = angle - 360
        
        if np.isnan(angle):
            angles.append(angle)
        elif len(angles) > 0:
            angles.append(pref_angle + angle)
            pref_angle = pref_angle + angle
        else:
            angles.append(angle)
            pref_angle = angle
        
        pref_vector = vector
        
    return angles

def plot_angles_histogram(dataset, axes=[0,1], directional=True):
    if directional:
        angles = angle_directional(dataset[:, 0, 0, :], dataset[:, 0, 1, :], axes)
    else:
        angles = calculate_angles(dataset[:, 0, 0, :], dataset[:, 0, 1, :], axes)
    plt.figure()
    plt.title('angle (degrees) to positive x-axis histogram')
    plt.hist(angles, bins=45)
    plt.show()
    return

# Tail stuff

In [ ]:
def make_dot(v1, v2):
    dot = 0
    for i in range(len(v1)):
        dot += v1[i] * v2[i]
    return dot

def make_mag(vector):
    mag2 = 0
    for i in vector:
        mag2 += i**2
    mag = np.sqrt(mag2)
    return mag

def angle_between(v1, v2):
    dot = make_dot(v1, v2)
    mag1 = make_mag(v1)
    mag2 = make_mag(v2)
    angle = math.degrees(math.acos(dot/(mag1*mag2)))
    return angle

def tail_angles(data):
    head = data[:, 0, 0, :]
    thorax = data[:, 0, 1, :]
    tail = data[:, 0, 2, :]
    angles = []
    for i in range(len(head)):
        vector_head_flip = [(thorax[i][0] - head[i][0]), (thorax[i][1] - head[i][1])]
        vector_head = [(head[i][0] - thorax[i][0]), (head[i][1] - thorax[i][1])]
        vector_tail = [(tail[i][0] - thorax[i][0]), (tail[i][1] - thorax[i][1])]
        angle_smallest = angle_between(vector_head_flip, vector_tail)
        angle_tail = calc_angle_xpos(vector_tail)
        angle_head = calc_angle_xpos(vector_head)
        
        diff = angle_tail - angle_head
        if diff < 0:
            angle = 180 + diff
        else:
            angle = diff - 180
        angles.append(angle)
        
    return angles

def histogram_tail_angles(dataset):
    plt.figure()
    plt.hist(tail_angles(dataset), bins=360)
    plt.xlim(-180, 180)
    plt.show()
    return

# z-angle with xy plane

In [ ]:
def z_xyplane_angle(data_1, data_2):
    angles = []
    for i in range(len(data_1)):
        angle = angle_between(data_1[i] - data_2[i], [data_1[i][0]-data_2[i][0], data_1[i][1]-data_2[i][1], 0])
        
        if data_1[i][2]-data_2[i][2] > 0:
            angles.append(angle)
        else: 
            angles.append(-angle)
    return angles

# Speed

In [ ]:
from numpy.polynomial import polynomial as P
#import multipolyfit.multipolyfit as mpf

In [ ]:
def calculate_poly_values(c, x_values):
    y_values = []
    for x in x_values:
        y = 0
        for i, const in enumerate(c):
            y += const * np.power(x, i)
        y_values.append(y)
    return y_values

def derivative_poly(c, x):
    derivative = 0
    for i in range(1, len(c)):
        derivative += c[i] * (i) * np.power(x, i-1) 
    return derivative

def velocity_single_direction(data_i, df, degrees):
    
    velocities = []
    for i in range(df, len(data_i) - df):
        data_r = data_i[i-df:i+df+1]
        c = P.polyfit(np.linspace(i - df, i - df + len(data_r) - 1, len(data_r)), data_r, degrees)
        velocities.append(derivative_poly(c, i) * 100)
    
    return velocities

In [ ]:
def speed_upanddown(speeds):
    speed_up_y = []
    speed_up_x = []
    speed_down_y = []
    speed_down_x = []

    for i in range(1, len(speeds) - 1):
        pref = speeds[i-1]
        now = speeds[i]
        later = speeds[i+1]

        if abs(now) > abs(pref):
            speed_up_x.append(i)
            speed_up_y.append(now)
        else:
            speed_down_x.append(i)
            speed_down_y.append(now)
            
    return [speed_up_x, speed_up_y], [speed_down_x, speed_down_y]

# Video xy positions heatmap moving frame region

In [ ]:
def make_frame(x, y, n):
    heatmap, xedges, yedges = np.histogram2d([i for i in y if not np.isnan(i)], [i for i in x if not np.isnan(i)], bins=(64,64))
    extent = [xedges[0], xedges[-1], yedges[0], yedges[-1]]
    
    plt.ioff()
    fig, ax = plt.subplots(figsize=(10,10))
    canvas = FigureCanvas(fig)
    
    ax.set_title('full experiment xy heatmap')
    ax.set_ylabel('y')
    ax.set_xlabel('x')
    ax.set_ylim(0, 40)
    ax.set_xlim(0, 40)
    plt.imshow(heatmap, extent=extent, origin='lower')
    plt.colorbar()
    
    canvas.draw()
    s, (width, height) = canvas.print_to_buffer()
    
    # convert the figure to an array of the correct size
    img = np.frombuffer(canvas.tostring_rgb(), dtype='uint8').reshape(width, height, 3)
    
    plt.close()
    plt.ion()
    
    return img

# Video Fish

In [ ]:
def animate_scatters(iteration, data, scatters, lines, scatter_projections, lines_projections, text, all_timestamps):
    """
    Update the data held by the scatter plot and therefore animates it.

    Args:
        iteration (int): Current iteration of the animation
        data (list): List of the data positions at each iteration.
        scatters (list): List of all the scatters (One per element)

    Returns:
        list: List of scatters (One per element) with new coordinates
    """
    #if np.mod(iteration, 100) == 0:
        #print(iteration)
    # parse the input shapes
    numFrames, numFish, numBodyPoints, _ = data.shape
 
    # update the scatters
    for fishIdx in range(numFish):
        for bpIdx in range(numBodyPoints):
            scatters[fishIdx][bpIdx]._offsets3d = (data[iteration,fishIdx,bpIdx,0:1],
                                                   data[iteration,fishIdx,bpIdx,1:2],
                                                   data[iteration,fishIdx,bpIdx,2:])
            
    # update the lines
    for fishIdx,line in enumerate(lines):
        line.set_data(data[iteration, fishIdx, :, :2].swapaxes(0,1))
        line.set_3d_properties(data[iteration, fishIdx, :, 2])
        
        
    # update the text
    text.set_text(all_timestamps[iteration])
        
    # -------------- #
    for projectionIdx in range(3):
        
        # XZ => Y=0
        if projectionIdx == 0:
            projectionData = np.copy(data[iteration])
            projectionData[:, :, 1] = 0
        # XY => Z=0
        elif projectionIdx == 1:
            projectionData = np.copy(data[iteration])
            projectionData[:, :, 2] = 0
        # YZ => X=0
        elif projectionIdx == 2:
            projectionData = np.copy(data[iteration])
            projectionData[:, :, 0] = 0
            
        proj_scatters = scatter_projections[projectionIdx]
        proj_lines = lines_projections[projectionIdx]
        
        # update the scatters
        for fishIdx in range(numFish):
            for bpIdx in range(numBodyPoints):
                proj_scatters[fishIdx][bpIdx]._offsets3d = (projectionData[fishIdx,bpIdx,0:1],
                                                            projectionData[fishIdx,bpIdx,1:2],
                                                            projectionData[fishIdx,bpIdx,2:])
        # update the lines
        for fishIdx,line in enumerate(proj_lines):
            line.set_data(projectionData[fishIdx, :, :2].swapaxes(0,1))
            line.set_3d_properties(projectionData[fishIdx, :, 2])

    return




def make_movie(data, f0, fE, savePath='/home/liam/temp/testanimation.mp4'):
    """
    Creates the 3D figure and animates it with the input data.

    Args:
        data (list): List of the data positions at each iteration.
        save (bool): Whether to save the recording of the animation. (Default to False).
    """
    data = data[f0:fE]
    
    # parse the input shapes
    numFrames, numFish, numBodyPoints, _ = data.shape
    fish_colors = ['red', 'blue']

    # Attaching 3D axis to the figure
    fig = plt.figure()
    fig.tight_layout()
    ax = p3.Axes3D(fig, [0.,0.,1.,1.])

    # Initialize scatters (list over fish, list over bps)
    symbols = ['o', 's', 'x']
    sizes=12
    
    # Main scatters
    scatters = []
    for fishIdx in range(numFish):
        col = fish_colors[fishIdx]
        fish_scatters = []
        for bpIdx in range(numBodyPoints):
            fish_scatters.append(ax.scatter(data[0,fishIdx,bpIdx,0:1], data[0,fishIdx,bpIdx,1:2], data[0,fishIdx,bpIdx,2:], 
                                            c=col, s=sizes, marker=symbols[bpIdx])) 
        scatters.append(fish_scatters)
        
    # Main lines
    lines = []
    for fishIdx in range(numFish):
        col = fish_colors[fishIdx]
        line = ax.plot(data[0, fishIdx, :, 0], data[0, fishIdx, :, 1], data[0, fishIdx, :, 2], c=col)[0]
        lines.append(line)
        
    # ----------------
    # projections
    symbols = ['x', 'o', 's']
    sizes=5

    scatter_projections = []
    lines_projections = []
    for projectionIdx in range(3):
        proj_scatters = []
        proj_lines = []
        for fishIdx in range(numFish):
            col = fish_colors[fishIdx]
            # scatter plots
            fish_scatters = []
            for bpIdx in range(numBodyPoints):
                #print(symbols[bpIdx])
                fish_scatters.append(ax.scatter(data[0,fishIdx,bpIdx,0:1], data[0,fishIdx,bpIdx,1:2], data[0,fishIdx,bpIdx,2:], 
                                                c=col, s=sizes, marker=symbols[bpIdx], alpha=0.1))
            # lines
            line = ax.plot(data[0, fishIdx, :, 0], data[0, fishIdx, :, 1], data[0, fishIdx, :, 2], c=col, alpha=0.1)[0]
            proj_lines.append(line)
            proj_scatters.append(fish_scatters)
        
        scatter_projections.append(proj_scatters)
        lines_projections.append(proj_lines)
    # ----------------
    
    # the text
    all_timestamps = ['frame: '+ str(i).zfill(7) for i in range(f0, fE)]
    time_text = ax.text(14, 0, 43,'', fontsize=15)

    # Setting the axes properties
    ax.set_xlim3d([0, 40])
    ax.set_xlabel('X')

    ax.set_ylim3d([0, 40])
    ax.set_ylabel('Y')

    ax.set_zlim3d([0, 40])
    ax.set_zlabel('Z')

    # Provide starting angle for the view
    elevation_angle = 10
    azimuthal_angle = 45
    ax.view_init(elevation_angle, azimuthal_angle)

    ani = animation.FuncAnimation(fig, animate_scatters, numFrames, fargs=(data, scatters, lines, 
                                                                           scatter_projections, lines_projections,
                                                                           time_text, all_timestamps),
                                       interval=10, blit=False, repeat=True)

    Writer = animation.writers['ffmpeg']
    writer = Writer(fps=100, metadata=dict(artist='LOS'), bitrate=3500)
    ani.save(savePath, writer=writer)
    return

# Running code

In [ ]:
solo_data = data_3D_filtered[0]

### Missing data

In [ ]:
# multiple
for dataset in data_3D_filtered:
    missing_data(dataset)
    print()

In [ ]:
# solo
missing_data(solo_data)

### Positions (plots=[scatter, histogram, heatmap])

In [ ]:
# multiple
for dataset in data_3D_filtered:
    bodypoint_positions(dataset, plots=[True, False, False], names=['x', 'y', 'z'], frame_range=None)

In [ ]:
# solo
bodypoint_positions(solo_data, plots=[True, True, True], names=['x', 'y', 'z'], frame_range=None)

In [ ]:
# heatmap z specific
z_specific_xy(solo_data[:, 0, 1, :], 0, 40, 5)

### Angles

##### Upperbody xy

In [ ]:
# directional (0 to 360 degrees)
angles_upperbody = angle_directional(solo_data[:, 0, 0, :], solo_data[:, 0, 1, :], [0,1])

In [ ]:
# additive [adds degrees relative to previous position ("remembers")]
angles_upperbody_additive = angle_add_xy(solo_data)

In [ ]:
plot_scatter_data(angles_upperbody_additive)

In [ ]:
# histogram of angles
plot_angles_histogram(solo_data, directional=True)

##### Upperbody z

In [ ]:
z_angles_upperbody = z_xyplane_angle(solo_data[:, 0, 0, :], solo_data[:, 0, 1, :])

In [ ]:
solo_data2 = data_3D_filtered[5]
z_angles_upperbody2 = z_xyplane_angle(solo_data2[:, 0, 0, :], solo_data2[:, 0, 1, :])

In [ ]:
plot_scatter_data(z_angles_upperbody)

In [ ]:
plt.figure()
plt.hist(z_angles_upperbody, bins=100, label="fish1", alpha=0.5)
plt.hist(z_angles_upperbody2, bins=100, label="fish2", alpha=0.5)
plt.axvline(0)
plt.show()

##### Tail xy

In [ ]:
# histogram for all
for dataset in data_3D_filtered:
    histogram_tail_angles(dataset)

In [ ]:
# histogram solo
histogram_tail_angles(solo_data)

In [ ]:
# scatterplot/data angles
angles_tail = tail_angles(solo_data)

In [ ]:
plot_scatter_data(angles_tail[479000:481000])

##### Tail z

In [ ]:
z_angles_tail = z_xyplane_angle(solo_data[:, 0, 2, :], solo_data[:, 0, 1, :])

In [ ]:
plot_scatter_data(z_angles_tail[530000:550000])

In [ ]:
plt.figure()
plt.hist(z_angles_tail, bins=100)
plt.show()

### Speed

##### Directional

In [ ]:
data = solo_data[:, 0, 1, :]
velocities_x = velocity_single_direction(data[:, 0], 5, 2)
velocities_y = velocity_single_direction(data[:, 1], 5, 2)
velocities_z = velocity_single_direction(data[:, 2], 5, 2)

# absolute speeds
speeds_directional = []
for i in range(len(velocities_x)):
    speeds_directional.append(np.sqrt(np.power(velocities_x[i], 2) + np.power(velocities_y[i], 2) + np.power(velocities_z[i], 2)))

In [ ]:
plt.figure()
plt.hist(speeds_directional, bins=100)
plt.show()

In [ ]:
speed_up, speed_down = speed_upanddown(speeds_directional[479000:481000])

plt.figure()
plt.scatter(*speed_up, label="acceleration")
plt.scatter(*speed_down, label="deceleration")
plt.legend()
plt.show()

In [ ]:
# one direction:
velocities_i = np.copy(velocities_z)

i_up, i_down = speed_upanddown(velocities_i[479000:481000])

plt.figure()
plt.scatter(*i_up, label="acceleration")
plt.scatter(*i_down, label="deceleration")
plt.legend()
plt.show()

##### Speed of anything

In [ ]:
# angles_upperbody_additive, z_angles_upperbody, angles_tail, z_angles_tail
dataset_something = np.copy(angles_tail)

speed_something = velocity_single_direction(dataset_something, 5, 2)

In [ ]:
plt.figure()
plt.hist(speed_something, bins=100)
plt.show()

In [ ]:
dataset_something2 = np.copy(angles_upperbody_additive)

speed_something2 = velocity_single_direction(dataset_something2, 5, 2)

In [ ]:
plt.figure()
plt.hist(speed_something2, bins=100)
plt.show()

### Make movie

In [ ]:
# make the movie

t0 = time.time()

# frame range
f0 = 250400
fE = 250600

make_movie(solo_data, f0, fE, savePath='./0_250400_250600.mp4')

tE = time.time()
print(tE-t0)